In [1]:
import xarray as xr
import numpy as np
import requests
import io

### Setup

In [17]:
def test_ensemble_mean(ds, lat, lon, models):

    for var in list(ds.data_vars):
        if var not in ["spatial_ref", "crs"]:
            print(f"Indicator: {var}")

            print("Original Dataset Values:")
            values = (
                ds[var]
                .sel(
                    # lat=lat,
                    # lon=lon,
                    method="nearest",
                )
                .sel(model=models)
                .values
            )

            print(values)
            print("\n Calculated Ensemble Mean:")
            # skip -9999 when calculating the mean
            values = np.where(values == -9999, np.nan, values)
            print(np.nanmean(values))

            print("Dataset Ensemble Mean:")
            print(
                ds[var]
                .sel(
                    # lat=lat,
                    # lon=lon,
                    method="nearest",
                )
                .sel(model=13)
                .values
            )

            print("\n\n")

In [22]:
coverage_id = "cmip6_monthly_cf_timefix"

# test will use this location
lat = 65
lon = -142
models = slice(0, 13)  # first 12 models - model 13 is the ensemble mean

vars = [
    "clt",
    # "evspsbl",  # evspsbl values are in scientific notation and do not encode correctly to netCDF
    "hfls",
    "hfss",
    "pr",
    "prsn",
    "psl",
    "rlds",
    "rsds",
    "sfcWind",
    "siconc",
    "snw",
    "tas",
    "tasmax",
    "tasmin",
    "ts",
    "uas",
    "vas",
]

# create comma separated string of variables
var_string = ",".join(vars)

# query only first and last timesteps / scenarios of the datacube
times = [0, 55121]
scenarios = [0, 4]

hist_url = f"https://zeus.snap.uaf.edu/rasdaman/ows?&SERVICE=WCS&VERSION=2.0.1&REQUEST=GetCoverage&COVERAGEID={coverage_id}&SUBSET=lat({lat})&SUBSET=lon({lon})&SUBSET=scenario({scenarios[0]})&SUBSET=time({times[0]})&RANGESUBSET={var_string}&FORMAT=application/netcdf"
proj_url = f"https://zeus.snap.uaf.edu/rasdaman/ows?&SERVICE=WCS&VERSION=2.0.1&REQUEST=GetCoverage&COVERAGEID={coverage_id}&SUBSET=lat({lat})&SUBSET=lon({lon})&SUBSET=scenario({scenarios[1]})&SUBSET=time({times[1]})&RANGESUBSET={var_string}&FORMAT=application/netcdf"

### Historical

In [19]:
with requests.get(hist_url) as response:
    print(hist_url)
    ds_hist = xr.open_dataset(
        io.BytesIO(response.content), decode_cf=True, mask_and_scale=False
    )
ds_hist

https://zeus.snap.uaf.edu/rasdaman/ows?&SERVICE=WCS&VERSION=2.0.1&REQUEST=GetCoverage&COVERAGEID=cmip6_monthly_cf_timefix&SUBSET=lat(65)&SUBSET=lon(-142)&SUBSET=scenario(0)&SUBSET=time(0)&RANGESUBSET=clt,hfls,hfss,pr,prsn,psl,rlds,rsds,sfcWind,siconc,snw,tas,tasmax,tasmin,ts,uas,vas&FORMAT=application/netcdf


<xarray.Dataset> Size: 1kB
Dimensions:      (model: 14)
Coordinates:
  * model        (model) float64 112B 0.0 1.0 2.0 3.0 ... 10.0 11.0 12.0 13.0
Data variables: (12/18)
    clt          (model) int32 56B ...
    hfls         (model) float32 56B ...
    hfss         (model) float32 56B ...
    pr           (model) float32 56B ...
    prsn         (model) float32 56B ...
    psl          (model) float32 56B ...
    ...           ...
    tasmax       (model) float32 56B ...
    tasmin       (model) float32 56B ...
    ts           (model) float32 56B ...
    uas          (model) float32 56B ...
    vas          (model) float32 56B ...
    spatial_ref  int32 4B ...
Attributes:
    Conventions:  CF-1.8
    contact:      uaf-snap-data-tools@alaska.edu
    description:  Monthly data from 13 CMIP6 models on a common grid, includi...
    institution:  Scenarios Network for Alaska and Arctic Planning, Universit...
    source:       CMIP6 model output
    title:        CMIP6 Monthly Data on a Common Grid with Multi-Model Ensemb...
    url:          https://uaf-snap.org/

In [20]:
test_ensemble_mean(ds_hist, lat, lon, models)

Indicator: clt
Original Dataset Values:
[87 79 98 91 92 84 94 96 49 79 60 86 92 84]

 Calculated Ensemble Mean:
83.64285714285714
Dataset Ensemble Mean:
84



Indicator: hfls
Original Dataset Values:
[-2.2817163  -0.65714633  0.5393715  -1.0443037   2.5695183  -0.3792504
 -0.3091934  -0.29590857  0.16051397  1.2170705   1.8568106  -2.2377229
 -0.680343   -0.11863844]

 Calculated Ensemble Mean:
-0.11863844
Dataset Ensemble Mean:
-0.11863844



Indicator: hfss
Original Dataset Values:
[-13.770391   -8.553142  -11.729184   -7.598542  -14.437607   -3.967609
  -3.5726087  -4.4132047 -22.338554  -13.087814  -27.308784  -11.839965
  -8.935531  -11.657918 ]

 Calculated Ensemble Mean:
-11.657918
Dataset Ensemble Mean:
-11.657918



Indicator: pr
Original Dataset Values:
[52.6 11.6 47.  35.9 33.2 30.2 12.3 68.6 30.1 52.3 37.8 37.4 18.4 35.9]

 Calculated Ensemble Mean:
35.95
Dataset Ensemble Mean:
35.9



Indicator: prsn
Original Dataset Values:
[52.6 11.5 46.4 35.8 32.1 30.1 12.2 67.9 29.  51

/var/folders/dz/1ccfqg_n5mg3fsrt8rwm7fd80000gn/T/ipykernel_2443/1089177118.py:23: RuntimeWarning: Mean of empty slice
  print(np.nanmean(values))


### Projected

In [23]:
with requests.get(proj_url) as response:
    print(proj_url)
    ds_proj = xr.open_dataset(
        io.BytesIO(response.content), decode_cf=True, mask_and_scale=False
    )
ds_proj

https://zeus.snap.uaf.edu/rasdaman/ows?&SERVICE=WCS&VERSION=2.0.1&REQUEST=GetCoverage&COVERAGEID=cmip6_monthly_cf_timefix&SUBSET=lat(65)&SUBSET=lon(-142)&SUBSET=scenario(4)&SUBSET=time(55121)&RANGESUBSET=clt,hfls,hfss,pr,prsn,psl,rlds,rsds,sfcWind,siconc,snw,tas,tasmax,tasmin,ts,uas,vas&FORMAT=application/netcdf


<xarray.Dataset> Size: 1kB
Dimensions:      (model: 14)
Coordinates:
  * model        (model) float64 112B 0.0 1.0 2.0 3.0 ... 10.0 11.0 12.0 13.0
Data variables: (12/18)
    clt          (model) int32 56B ...
    hfls         (model) float32 56B ...
    hfss         (model) float32 56B ...
    pr           (model) float32 56B ...
    prsn         (model) float32 56B ...
    psl          (model) float32 56B ...
    ...           ...
    tasmax       (model) float32 56B ...
    tasmin       (model) float32 56B ...
    ts           (model) float32 56B ...
    uas          (model) float32 56B ...
    vas          (model) float32 56B ...
    spatial_ref  int32 4B ...
Attributes:
    Conventions:  CF-1.8
    contact:      uaf-snap-data-tools@alaska.edu
    description:  Monthly data from 13 CMIP6 models on a common grid, includi...
    institution:  Scenarios Network for Alaska and Arctic Planning, Universit...
    source:       CMIP6 model output
    title:        CMIP6 Monthly Data on a Common Grid with Multi-Model Ensemb...
    url:          https://uaf-snap.org/

In [24]:
test_ensemble_mean(ds_proj, lat, lon, models)

Indicator: clt
Original Dataset Values:
[   91    91 -9999    93    95    97    94    97    61    82    93    81
    90    89]

 Calculated Ensemble Mean:
88.76923076923077
Dataset Ensemble Mean:
89



Indicator: hfls
Original Dataset Values:
[-1.7992208  -1.601384           nan -1.7520978   4.4892306   0.6637722
 -1.0122594  -0.24933378  2.5591097   0.77363914  5.3989015   0.14935197
 -1.6268543   0.49940455]

 Calculated Ensemble Mean:
0.4994046
Dataset Ensemble Mean:
0.49940455



Indicator: hfss
Original Dataset Values:
[-12.357898  -11.768266          nan -11.285135  -19.808771   -3.2877831
  -4.0491204  -3.3174973 -21.357645  -15.635836  -17.491177  -24.14888
 -15.290747  -13.316563 ]

 Calculated Ensemble Mean:
-13.316563
Dataset Ensemble Mean:
-13.316563



Indicator: pr
Original Dataset Values:
[80.2 24.8  nan 24.6 41.9 50.8 35.2 32.3 48.5 66.9 98.7 66.  33.8 50.3]

 Calculated Ensemble Mean:
50.307686
Dataset Ensemble Mean:
50.3



Indicator: prsn
Original Dataset Values:
[80

/var/folders/dz/1ccfqg_n5mg3fsrt8rwm7fd80000gn/T/ipykernel_2443/1089177118.py:23: RuntimeWarning: Mean of empty slice
  print(np.nanmean(values))
